In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

In [ ]:
RUN = "dyntrans_full_3h_ws"
PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
INPUT = PREFIX + "results/" + RUN + "/integrated_comp/stats.csv"
OUTPUT = PREFIX + "results/" + RUN + "/graphics/pricecuts/"

In [ ]:
stats = pd.read_csv(INPUT, index_col=0)

In [ ]:
stats.head()

In [ ]:
def get_data(export):
    mg_AC_incl_H2 = stats[stats.index == export].loc[:,"mg_AC_inclu_H2 El_all"]
    mg_AC_exclu_H2 = stats[stats.index == export].loc[:,"mg_AC_exclu_H2 El_all"]
    mg_AC_False_False = stats[stats.index == export].loc[:,"mg_AC_False_False_all"]
    return mg_AC_incl_H2, mg_AC_exclu_H2, mg_AC_False_False 

In [ ]:
opts = stats[stats.index == 40].opts.str[4:]
opts = np.unique(
        opts.fillna(100).astype(float)
    )  # TODO improve the fillna value
opts_reverse = 1 - opts
opts_reverse[opts_reverse < 0] = 0
opts = opts_reverse[::-1] * 100

In [ ]:
# Plot the lcoe_w_electrolysis lcoe_w_no_electrolysis along the stats["opts"]


fig, ax = plt.subplots(figsize=(6, 3))

for export in [100]: # [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]:
    contour_mg_AC_incl_H2, contour_mg_AC_exclu_H2, contour_mg_AC_False_False = get_data(export)
    ax.plot(opts, contour_mg_AC_incl_H2, label="Export (w/ electrolysis)", color = "black", alpha = (export/300)+0.2)
    ax.plot(opts, contour_mg_AC_exclu_H2, label="Local (w/o electrolysis)", color = "green", alpha = (export/300)+0.2)
    ax.plot(opts, contour_mg_AC_False_False, label="Local-Export", color = "red", alpha = (export/300)+0.2)
    ax.set_label("Export in % of base levels")

ax.set_xlabel("Domestic mitigation in %")
ax.set_ylabel("Marginal price of electricity in €/MWh")
ax.set_xticks(opts)
ax.set_ylim(0, 100)
ax.set_xlim(0, 90)

ax.legend()

#ax.legend()
if not os.path.isdir(OUTPUT):
    os.makedirs(OUTPUT)
    print("Created directory " + OUTPUT)
plt.savefig(OUTPUT + "electricity" + str(export) + ".pdf", bbox_inches="tight")
plt.show()

Probability experiments

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Your existing code to get_data and other settings

fig, ax = plt.subplots(figsize=(12, 6))

exports = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]

for i, export in enumerate(exports):
    lcoe_w_electrolysis, lcoe_w_no_electrolysis, lcoe_w = get_data(export)
    
    ax.plot(opts, lcoe_w_electrolysis, color="black", alpha=(export / 300) + 0.2)
    ax.plot(opts, lcoe_w_no_electrolysis, color="green", alpha=(export / 300) + 0.2)
    ax.plot(opts, lcoe_w, color="red", alpha=(export / 300) + 0.2)

    # Fill between curves at each x point
    ax.fill_between(opts, lcoe_w_electrolysis, lcoe_w_no_electrolysis, color="black", alpha=0.2)
    ax.fill_between(opts, lcoe_w_no_electrolysis, lcoe_w, color="green", alpha=0.2)

ax.set_xlabel("Domestic mitigation in %")
ax.set_ylabel("Marginal price of electricity in €/MWh")
ax.set_xticks(opts)
ax.set_ylim(0, 100)
ax.set_xlim(0, 90)

# Add legend for the filled areas
ax.legend(["Export (w/ electrolysis)", "Local (w/o electrolysis)", "Local-Export", "Export to Local (w/o electrolysis)", "Local-Export to Local (w/o electrolysis)"])

if not os.path.isdir(OUTPUT):
    os.makedirs(OUTPUT)
    print("Created directory " + OUTPUT)
plt.savefig(OUTPUT + "hydrogen.pdf", bbox_inches="tight")
plt.show()
